In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from numpy.random import Generator, PCG64

G = Generator(PCG64())


n = 100

def stick_breaking(n: int, alpha: float) -> np.ndarray:
    """
    Draws n samples from a stick-breaking process with beta distribution intensity alpha.

    :param n: number of samples
    :param alpha: intensity parameter of the beta distribution
    :returns: stick lengths
    """
    betas = G.beta(a=1.0, b=alpha, size=n)
    betas[1:] *= np.cumprod(1.0 - betas[:-1])
    weights = np.sort(betas)[::-1]
    return weights

def mandelbrot(x: np.ndarray, B: float, c: float):
    y = 1.0 / np.power(c + x, B)
    y /= np.sum(y)
    assert np.isclose(1.0, np.sum(y), rtol=0.0, atol=1E-8)
    return y

def plot_stick_lengths(stick_lengths: np.ndarray, alpha: float, B: int) -> None:
    """
    Plots -log2(sticks)
    :param sticks: list of stick lenghts
    """
    n = len(stick_lengths)
    fig = plt.figure(figsize=(16, 9), dpi=400)
    subplot = fig.add_subplot(111, facecolor='white')
    subplot.plot(np.arange(n), stick_lengths, label=str('-log2(stick_lengths)'))
    plt.legend()
    plt.show()

In [ ]:
from numpy.linalg import norm

def zipf_law_norm(sample: np.ndarray, ord=None) -> float:
    n = len(sample)
    x = sample / sample[0] * (1 + np.arange(n))
    return norm(x - np.ones(n))

assert zipf_law_norm(np.array([24, 12, 8, 6, 4.8, 4])) == 1.1102230246251565e-16

In [ ]:
from sys import float_info

def grid_alpha_sample(start: float, stop: float, step: float,
        num_iter: int) -> float:
    min_sample_norm = float_info.max
    for alpha in np.arange(start, stop, step):
        for _ in range(num_iter):
            sample = stick_breaking(n, alpha)
            sample_norm = zipf_law_norm(sample)
            if min_sample_norm > sample_norm:
                min_sample_norm = sample_norm
                min_alpha, min_sample = alpha, sample
    return min_alpha, min_sample

num_iter = 10000
step = 1.0 / 8.0
test_alpha, test_sample = grid_alpha_sample(step, 250, step, num_iter)
indent, step = 25, 1.0 / 16.0
alpha, sample = grid_alpha_sample(
    max(step, test_alpha - indent), test_alpha + indent, step, num_iter)
print(alpha)

In [ ]:
fig = plt.figure(figsize=(16, 9), dpi=400)
subplot = fig.add_subplot(111, facecolor='white', title='stick_lengths')
subplot.plot(np.arange(n), stick_breaking(n, alpha), label=str(alpha))
plt.legend()
plt.show()

In [ ]:
def grid_c_B(sample: np.ndarray,
        start_B: float, stop_B: float, step_B: float,
        start_c: float, stop_c: float, step_c: float) -> tuple:
    if np.any(np.diff(sample) > 0):
        raise ValueError()
    min_sample_norm = float_info.max
    for B in np.arange(start_B, stop_B, step_B):
        for c in np.arange(start_c, stop_c, step_c):
            sample_norm = norm(mandelbrot(sample, B, c) - sample)
            if min_sample_norm > sample_norm:
                min_sample_norm = sample_norm
                min_B, min_c = B, c
    return min_B, min_c

In [ ]:
c, B = grid_c_B(sample, 1.0, 4.0, 1.0 / 256.0, -100.0, 100.0, 1.0 / 16.0)
print(c, B)

In [ ]:
stick_lengths = stick_breaking(n, alpha)
plot_stick_lengths(stick_lengths, alpha, B)